<a href="https://colab.research.google.com/github/yungminSW/study_test/blob/main/Multiple_linearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
# Load dataset
from sklearn.datasets import load_boston 
boston_ds = load_boston()

In [ ]:
frame = pd.DataFrame(boston_ds.data)
frame_ = pd.DataFrame(boston_ds.data, columns = boston_ds.feature_names)
# add target 
frame['target'] = boston_ds.target
frame_['target'] = boston_ds.target
print(frame_)

In [ ]:
# number of data points
N = len(frame[0])
# dimension of input space
dim = 13
# make constant matrix X
X = np.reshape(np.ones_like(frame[0]), (N,1))
for i in range(dim):
  X = np.concatenate((X, np.reshape(np.array(frame[i]), (N,1))), axis = 1)
print('X.shape = ', X.shape)
print(X)

In [ ]:
# LSE for B in the model y = XB + e
y = np.reshape(np.array(frame['target']), (N,1))
B_hat = np.matmul(np.matmul(np.linalg.inv((np.matmul(X.T,X))), X.T), y)
print('estimated B = ')
print(B_hat)

# predicted y
y_hat = np.matmul(X, B_hat)

# estimator for std(y)
s = math.sqrt(np.sum(np.power(y - np.matmul(X,B_hat),2)) / (N - dim - 1))

# estimator for variance of B_hat
cov_B = s**2 * np.linalg.inv((np.matmul(X.T,X)))
var_B_hat = cov_B.diagonal()
std_B_hat = np.sqrt(var_B_hat)
print('estimated VAR[B] = ')
print(np.reshape(var_B_hat, (dim+1,1)))

# get adjusted R^2
SSR = np.sum(np.power(y_hat - y,2))
SST = np.sum(np.power(y- np.mean(y),2))
R_2 = SSR/SST
adj_R_2 = ((N - 1)*(1 - R_2)) / (N - dim - 1)
print('Adusted R^2  = ', adj_R_2)

In [ ]:
def t_distribution(dof):
  end = 10
  N = 1000
  t = np.linspace(-end, end, N)
  if dof <= 100 :
    T = (math.gamma((dof+1)/2) / (math.gamma(dof/2)*math.sqrt(math.pi * dof))) \
      * np.power((1 + np.power(t,2)/dof), -(dof + 1) / 2.)
  
  # If dof is large enough, then T(dof) = N(0.1)
  else:
    mu = 0.
    var = 1.0
    T = (1 / np.sqrt(2 * np.pi * var**2)) * np.exp(-(t-mu)**2 / (2 * var**2))
  return (2 * end / N),t, T

# get t-distribution 
delta, t, t_dist = t_distribution(N - dim - 1)

'''Hypothesis test'''
# observed t
O_t = np.divide(np.squeeze(B_hat), std_B_hat)
print('Observed t-value = ')
print(np.reshape(O_t, (dim+1,1)))
# get P-value
P = np.zeros_like(O_t)
for i in range(len(O_t)):
  result = np.where(np.abs(t) > abs(O_t[i]), t_dist, np.zeros_like(t))
  P[i] = np.sum(result) * delta
print('P-value = ')
print(np.reshape(P, (dim+1,1)))


In [ ]:
plt.scatter(frame_['INDUS'], frame_['target'], alpha=0.4)

In [ ]:
plt.scatter(frame_['AGE'], frame_['target'], alpha=0.4)

In [ ]:
plt.scatter(frame_['RM'], frame_['target'], alpha=0.4)